## обработка экспериментов

In [87]:
import pandas as pd

df = pd.read_csv('exp_res/experiments_results1.csv')

df.columns = ["mu_h_w", "mu_h_w_b", "mu_o_w",
                "glad_w", "glad_w_b", "alfa_w",
                "A_w", "A_end_w", "freq_w", "alfa_o",
                "A_o", "A_end_o", "freq_o", 'Q']

df['Q'] = df['Q'].where(df["Q"]==0.3, df["Q"]*86400)
df['mu_otn'] = df['mu_h_w'] / df['mu_h_w_b']

In [ ]:
def remove_outliers_iqr(df, columns):
    cleaned_df = df.copy()
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.9 * IQR
        upper_bound = Q3 + 3.5 * IQR
        cleaned_df = cleaned_df[(cleaned_df[col] >= lower_bound) & (cleaned_df[col] <= upper_bound)]
    return cleaned_df

df_new = df[(df['alfa_w']>0) & (df['alfa_o']>0)]

df_new = remove_outliers_iqr(df_new, ['freq_o', 'freq_w'])

df_new['Q'].hist()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = df_new

# Выбираем интересующие нас параметры
params = ["mu_otn", "mu_o_w", "glad_w", "glad_w_b", 'Q']
targets = ["alfa_w", "A_w", "freq_w", "alfa_o", "A_o", "freq_o"]

# Строим корреляционную матрицу
corr_matrix = df[params + targets].corr().loc[params, targets]

# Визуализируем корреляцию
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Корреляция между входными и выходными параметрами")
plt.show()


In [ ]:
sns.pairplot(df, vars=params + targets, diag_kind="kde")
plt.show()

In [ ]:
# sns.pairplot(df, vars=params + targets, diag_kind="kde")
sns.pairplot(df, x_vars=targets, y_vars=params, diag_kind=None)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Фиксируем размер графиков
plt.figure(figsize=(12, 8))

# Обучаем модель для каждого таргета
for i, target in enumerate(targets, 1):
    X = df[params]  # Входные параметры
    y = df[target]  # Выходная переменная
    
    # Разделение данных на train и test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Обучаем модель RandomForest
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)
    
    # Оцениваем важность признаков
    importances = pd.Series(rf.feature_importances_, index=params)
    
    # Визуализируем важность признаков
    plt.subplot(2, 3, i)  # Размещаем графики в сетке 3x2
    importances.sort_values().plot(kind="barh", title=f"Важность признаков для {target}")
    plt.xlabel("Значимость")
    
plt.tight_layout()  # Делаем графики компактными
plt.show()
